<a href="https://colab.research.google.com/github/zizoupavon/Deep-Learning---LLM-vs-Fine-Tuned-SLM-with-LoRA/blob/main/Project_Intent_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("bitext/bitext-gen-ai-chatbot-customer-support-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'bitext-gen-ai-chatbot-customer-support-dataset' dataset.
Path to dataset files: /kaggle/input/bitext-gen-ai-chatbot-customer-support-dataset


## Summary:

### Q&A
Yes, the dataset has been successfully loaded into the `df` DataFrame.

### Data Analysis Key Findings
*   The primary CSV file, `Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv`, was successfully identified and loaded into a pandas DataFrame named `df`.
*   The DataFrame `df` contains columns such as 'flags', 'instruction', 'category', 'intent', and 'response', as verified by displaying its head.

### Insights or Next Steps
*   The loaded DataFrame `df` is now ready for further data exploration, cleaning, and analysis to understand the customer support interactions.
*   A logical next step would be to perform a quick data overview, including checking data types, missing values, and unique values in categorical columns.


In [4]:
import pandas as pd
import os

# List files in the dataset directory
files_in_path = os.listdir(path)
print(f"Files available in the dataset directory: {files_in_path}")

# Identify the main CSV file. Based on previous runs or common dataset structures,
# it's often 'Bitext_Sample_-_es_to_en_-_Sheet1.csv'
# Let's try to find a .csv file or assume the given name.
csv_file_name = None
for file in files_in_path:
    if file.endswith('.csv'):
        csv_file_name = file
        break

if csv_file_name is None:
    print("No CSV file found in the dataset directory. Attempting to use a known filename.")
    # Fallback to a known filename if no CSV is automatically detected
    csv_file_name = 'Bitext_Sample_-_es_to_en_-_Sheet1.csv'

csv_file_path = os.path.join(path, csv_file_name)

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

print(f"Successfully loaded '{csv_file_name}' into DataFrame 'df'.")

# Display the head of the DataFrame to verify
df.head()

Files available in the dataset directory: ['Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv']
Successfully loaded 'Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv' into DataFrame 'df'.


,flags,instruction,category,intent,response
0,B,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,BQZ,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,BLQZ,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,BL,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,BCELN,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26872 entries, 0 to 26871
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   flags        26872 non-null  object
 1   instruction  26872 non-null  object
 2   category     26872 non-null  object
 3   intent       26872 non-null  object
 4   response     26872 non-null  object
dtypes: object(5)
memory usage: 1.0+ MB


In [6]:
import re

# Define the regex pattern for placeholders
placeholder_pattern = r"\{\{([^}]+)\}\}"

# Initialize a set to store unique placeholders
unique_placeholders = set()

# Iterate through the 'instruction' column and extract placeholders
if 'instruction' in df.columns:
    for instruction_text in df['instruction'].astype(str):
        matches = re.findall(placeholder_pattern, instruction_text)
        for match in matches:
            unique_placeholders.add(match.strip())

    print("Unique placeholders found:")
    for placeholder in sorted(list(unique_placeholders)):
        print(placeholder)
else:
    print("The 'instruction' column does not exist in the DataFrame.")

Unique placeholders found:
Account Category
Account Type
Currency Symbol
Delivery City
Delivery Country
Invoice Number
Order Number
Person Name
Refund Amount


In [7]:
import re

# Define the mapping of placeholders to synthetic values
synthetic_values = {
    "Order Number": "ORD-2025-12345",
    "Person Name": "Maria Garcia",
    "email": "alex.smith@example.com", # Assuming 'email' is a placeholder
    "product_name": "Wireless Mouse", # Assuming 'product_name' is a placeholder
    "date": "05/09/2025", # Assuming 'date' is a placeholder
    "Refund Amount": "$50.00", # Based on previously found placeholders
    "Account Category": "Savings", # Based on previously found placeholders
    "Account Type": "Checking", # Based on previously found placeholders
    "Currency Symbol": "$", # Based on previously found placeholders
    "Delivery City": "New York", # Based on previously found placeholders
    "Delivery Country": "USA", # Based on previously found placeholders
    "Invoice Number": "INV-2025-54321", # Based on previously found placeholders
}

# Create a copy of the DataFrame to store the modified instructions
df_synthetic = df.copy()

# Function to replace placeholders
def replace_placeholders(text):
    for placeholder, value in synthetic_values.items():
        # Use re.escape to handle special characters in placeholder names
        text = re.sub(r'\{\{' + re.escape(placeholder) + r'\}\}', value, text, flags=re.IGNORECASE)
    return text

# Apply the replacement function to the 'instruction' column
if 'instruction' in df_synthetic.columns:
    df_synthetic['instruction_synthetic'] = df_synthetic['instruction'].apply(replace_placeholders)
    print("Instructions with synthetic values generated successfully. Displaying original and synthetic instructions for verification:")
    display(df_synthetic[['instruction', 'instruction_synthetic']].head())
else:
    print("The 'instruction' column does not exist in the DataFrame.")

Instructions with synthetic values generated successfully. Displaying original and synthetic instructions for verification:


,instruction,instruction_synthetic
0,question about cancelling order {{Order Number}},question about cancelling order ORD-2025-12345
1,i have a question about cancelling oorder {{Or...,i have a question about cancelling oorder ORD-...
2,i need help cancelling puchase {{Order Number}},i need help cancelling puchase ORD-2025-12345
3,I need to cancel purchase {{Order Number}},I need to cancel purchase ORD-2025-12345
4,"I cannot afford this order, cancel purchase {{...","I cannot afford this order, cancel purchase OR..."


In [8]:
df_synthetic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26872 entries, 0 to 26871
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   flags                  26872 non-null  object
 1   instruction            26872 non-null  object
 2   category               26872 non-null  object
 3   intent                 26872 non-null  object
 4   response               26872 non-null  object
 5   instruction_synthetic  26872 non-null  object
dtypes: object(6)
memory usage: 1.2+ MB


In [9]:
if 'instruction_synthetic' in df_synthetic.columns:
    print("Sampling 20 processed rows for verification:")
    display(df_synthetic[['instruction', 'instruction_synthetic']].sample(n=20, random_state=42))
else:
    print("The 'instruction_synthetic' column does not exist in the DataFrame. Please ensure the placeholder replacement step was executed correctly.")

Sampling 20 processed rows for verification:


,instruction,instruction_synthetic
9329,I can't talk with a human agent,I can't talk with a human agent
4160,I have got to locate hte bills from {{Person N...,I have got to locate hte bills from Maria Garcia
18500,"I cannot pay, help me to inform of a problem w...","I cannot pay, help me to inform of a problem w..."
8840,I want help speaking to customer service,I want help speaking to customer service
5098,I try to see th accepted payment options,I try to see th accepted payment options
17250,where to sign up to the company nmewsletter,where to sign up to the company nmewsletter
3589,I'd like to see the withdrwaal fee how can i d...,I'd like to see the withdrwaal fee how can i d...
9043,I want to speak with someone,I want to speak with someone
15800,can you help me getting bill #85632?,can you help me getting bill #85632?
4384,I don't know how to take a quick look at invoi...,I don't know how to take a quick look at invoi...


In [10]:
from sklearn.model_selection import train_test_split

# Drop the original 'instruction' column and keep 'instruction_synthetic'
if 'instruction' in df_synthetic.columns:
    df_synthetic = df_synthetic.drop(columns=['instruction'])

# Define the features (X) and target (y) if applicable, or just the dataframe for splitting
X = df_synthetic.drop(columns=['response'])
y = df_synthetic['response']

# First split: 80% train, 20% (validation + test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)

# Second split: 50% validation, 50% test from the 20% temporary set
# This results in 10% validation and 10% test from the original dataset
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Training set size: {len(X_train)} samples")
print(f"Validation set size: {len(X_val)} samples")
print(f"Test set size: {len(X_test)} samples")

# Display the head of the training set features as an example
print("\nHead of X_train:")
display(X_train.head())

Training set size: 21497 samples
Validation set size: 2687 samples
Test set size: 2688 samples

Head of X_train:


,flags,category,intent,instruction_synthetic
3566,BLQ,CANCEL,check_cancellation_fee,want help seeing the early termination fee
14625,BILMQ,ACCOUNT,edit_account,can ya help me updating the details included o...
6599,BLQZ,REFUND,check_refund_policy,want assistance checking your reimbursement plicy
9532,BIL,CONTACT,contact_human_agent,how do I speak to an assistant?
24355,BIZ,ACCOUNT,switch_account,can I use the Savings acclunt?


In [11]:
# Function to concatenate features into a single string
def create_combined_instruction(row):
    category = row['category'] if 'category' in row else ''
    intent = row['intent'] if 'intent' in row else ''
    instruction_synthetic = row['instruction_synthetic'] if 'instruction_synthetic' in row else ''

    # Format as desired, e.g., '[CATEGORY: <value>] [INTENT: <value>] <instruction>'
    return f"[CATEGORY: {category}] [INTENT: {intent}] {instruction_synthetic}"

# Apply to X_train
X_train['combined_instruction'] = X_train.apply(create_combined_instruction, axis=1)
print("X_train 'combined_instruction' head:")
display(X_train[['category', 'intent', 'instruction_synthetic', 'combined_instruction']].head())

# Apply to X_val
X_val['combined_instruction'] = X_val.apply(create_combined_instruction, axis=1)
print("\nX_val 'combined_instruction' head:")
display(X_val[['category', 'intent', 'instruction_synthetic', 'combined_instruction']].head())

# Apply to X_test
X_test['combined_instruction'] = X_test.apply(create_combined_instruction, axis=1)
print("\nX_test 'combined_instruction' head:")
display(X_test[['category', 'intent', 'instruction_synthetic', 'combined_instruction']].head())

X_train 'combined_instruction' head:


,category,intent,instruction_synthetic,combined_instruction
3566,CANCEL,check_cancellation_fee,want help seeing the early termination fee,[CATEGORY: CANCEL] [INTENT: check_cancellation...
14625,ACCOUNT,edit_account,can ya help me updating the details included o...,[CATEGORY: ACCOUNT] [INTENT: edit_account] can...
6599,REFUND,check_refund_policy,want assistance checking your reimbursement plicy,[CATEGORY: REFUND] [INTENT: check_refund_polic...
9532,CONTACT,contact_human_agent,how do I speak to an assistant?,[CATEGORY: CONTACT] [INTENT: contact_human_age...
24355,ACCOUNT,switch_account,can I use the Savings acclunt?,[CATEGORY: ACCOUNT] [INTENT: switch_account] c...



X_val 'combined_instruction' head:


,category,intent,instruction_synthetic,combined_instruction
6492,REFUND,check_refund_policy,can I see under which circumstances can I ask ...,[CATEGORY: REFUND] [INTENT: check_refund_polic...
5676,PAYMENT,check_payment_methods,I do not know how I can see the accepted payme...,[CATEGORY: PAYMENT] [INTENT: check_payment_met...
16372,REFUND,get_refund,where to obtain a refund of my money?,[CATEGORY: REFUND] [INTENT: get_refund] where ...
10163,ACCOUNT,create_account,i dont have a Savings account eyt i need to si...,[CATEGORY: ACCOUNT] [INTENT: create_account] i...
10312,ACCOUNT,create_account,where do i open a gold account for my fiance,[CATEGORY: ACCOUNT] [INTENT: create_account] w...



X_test 'combined_instruction' head:


,category,intent,instruction_synthetic,combined_instruction
15763,INVOICE,get_invoice,i do not know what i have to do to get my invo...,[CATEGORY: INVOICE] [INTENT: get_invoice] i do...
9003,CONTACT,contact_human_agent,transfer to live agent,[CATEGORY: CONTACT] [INTENT: contact_human_age...
6913,REFUND,check_refund_policy,I need help to see how long refunds take,[CATEGORY: REFUND] [INTENT: check_refund_polic...
8094,CONTACT,contact_customer_service,help to pseak to customer support,[CATEGORY: CONTACT] [INTENT: contact_customer_...
23963,ACCOUNT,switch_account,i do not know what to do to swtch to the premi...,[CATEGORY: ACCOUNT] [INTENT: switch_account] i...


Projects Goal is to -
Build, evaluate, and compare two modeling pathways for intent classification —
LLM-based zero/few-shot inference and fine-tuned transformers; both using uniform
evaluation metrics (Accuracy, Macro-F1, and Cost).
Your goal is to determine which pathway offers the best balance of performance, scalability,
and business value.

Approach 1: Zero- or Few-Shot via Large Language Models

Since your API key is already securely set up in secrets, we can proceed directly with sampling the data. I'll sample up to 10 examples from each unique intent class from your X_test and y_test datasets. This sampled data will be stored in X_test_sampled and y_test_sampled for all your LLM experiments.

Sample Test Data for LLM Experiments: Sample up to 10 examples from each unique intent class from the X_test and y_test datasets, storing them in X_test_sampled and y_test_sampled respectively. This ensures a controlled and representative dataset for LLM experiments while managing API costs.

Define Zero-Shot Prompt Structure: Create a Python function to construct the zero-shot prompt for the LLM. This prompt will use the combined_instruction from the sampled data and explicitly instruct the LLM to classify the intent, potentially listing the available intent classes to guide its response.

Implement LLM Inference Function: Provide a placeholder or example function that demonstrates how to make an LLM API call (e.g., using OpenAI's API client). This function will take a prompt and return the LLM's response. It will also include guidance for the user to insert their actual API key and handle the specific API integration for their chosen LLM, emphasizing careful usage to manage costs.

Run LLM Inference and Collect Predictions: Iterate through the sampled test data, generate a zero-shot prompt for each entry, call the LLM inference function to get a prediction, and collect all predicted intents. This step will also include parsing the LLM's response to extract the intent label.

Evaluate LLM Performance: Calculate and display the Accuracy and Macro-F1 scores by comparing the collected LLM predictions against the true intent labels from the sampled test data. This will provide the initial performance metrics for the zero-shot approach.

Final Task: Summarize the results of the zero-shot LLM intent classification and prepare for the next steps, potentially exploring few-shot examples or moving to the fine-tuned transformer approach.



# Task
Prepare the LLM environment, sample 10 examples from each unique intent class from `X_test` and `y_test` for LLM experiments, and then implement a zero-shot intent classification workflow. This workflow should include defining a prompt structure using `combined_instruction`, developing a function for LLM inference, running the inference to collect predictions, and finally evaluating the LLM's performance using Accuracy and Macro-F1 scores.

## Prepare LLM Environment and Sample Data

### Subtask:
Guide the user to set up their chosen LLM API key (e.g., as an environment variable) and then sample 10 examples from each unique intent class from the `X_test` and `y_test` datasets. This sampled data will be used for all LLM experiments to control costs.


### Set up LLM API Key

Before proceeding with LLM experiments, you'll need to set up your chosen LLM API key. This is typically done by setting it as an environment variable to keep your sensitive information secure and out of your code.

**Instructions:**

1.  **For local development or shell environment:** You can set an environment variable using `export` in your terminal:
    ```bash
    export OPENAI_API_KEY='your_openai_api_key_here'
    # Or for other models, e.g., Google's PaLM:
    export GOOGLE_API_KEY='your_google_api_key_here'
    ```
    Replace `'your_openai_api_key_here'` or `'your_google_api_key_here'` with your actual API key.

2.  **Within a Python script or Colab notebook:** You can set the environment variable programmatically using the `os` module. It's good practice to load this from a secret management service or, for simpler cases, directly within your code (though be cautious not to commit your actual keys).
    ```python
    import os
    # Example for OpenAI
    os.environ['OPENAI_API_KEY'] = 'your_openai_api_key_here'

    # Example for Google's PaLM
    # os.environ['GOOGLE_API_KEY'] = 'your_google_api_key_here'
    ```
    Remember to replace the placeholder with your actual API key.

**Note:** If you are using a service like Google Colab, you might use the 'Secrets' tab to securely store and access your API keys, rather than hardcoding them in the notebook.

**Reasoning**:
Now that the instructions for setting up the API key are provided, the next step is to implement the sampling logic. This involves identifying unique intents, sampling up to 10 examples per intent from the test sets, and combining them into new sampled DataFrames for features and labels.



# Task
```python
import pandas as pd

# Ensure 'intent' column is available in X_test for sampling
if 'intent' not in X_test.columns:
    print("Error: 'intent' column not found in X_test. Please ensure it's present for stratified sampling.")
else:
    # Get unique intent classes
    unique_intents = X_test['intent'].unique()
    print(f"Found {len(unique_intents)} unique intent classes.")

    X_test_sampled_list = []
    y_test_sampled_list = []

    # Iterate through each unique intent to sample examples
    for intent_class in unique_intents:
        # Filter X_test and y_test for the current intent class
        X_intent = X_test[X_test['intent'] == intent_class]
        y_intent = y_test[X_test['intent'] == intent_class] # y_test should align with X_test indices

        # Sample up to 10 examples from the current intent class
        # Use min(len(X_intent), 10) to handle classes with fewer than 10 examples
        if not X_intent.empty:
            sampled_indices = X_intent.sample(n=min(len(X_intent), 10), random_state=42).index
            X_test_sampled_list.append(X_test.loc[sampled_indices])
            y_test_sampled_list.append(y_test.loc[sampled_indices])

    # Concatenate all sampled data into new DataFrames
    X_test_sampled = pd.concat(X_test_sampled_list)
    y_test_sampled = pd.concat(y_test_sampled_list)

    print(f"\nTotal sampled examples: {len(X_test_sampled)}")
    print(f"Number of unique intents in sampled data: {X_test_sampled['intent'].nunique()}")

    print("\nHead of X_test_sampled:")
    display(X_test_sampled.head())

    print("\nValue counts for 'intent' in X_test_sampled to verify distribution:")
    display(X_test_sampled['intent'].value_counts())

    print("\nHead of y_test_sampled:")
    display(y_test_sampled.head())
```

Sample Test Data for LLM Experiments: Sample up to 10 examples from each unique intent class from the X_test and y_test datasets, storing them in X_test_sampled and y_test_sampled respectively. This ensures a controlled and representative dataset for LLM experiments while managing API costs.

In [12]:
import pandas as pd

# Ensure 'intent' column is available in X_test for sampling
if 'intent' not in X_test.columns:
    print("Error: 'intent' column not found in X_test. Please ensure it's present for stratified sampling.")
else:
    # Get unique intent classes
    unique_intents = X_test['intent'].unique()
    print(f"Found {len(unique_intents)} unique intent classes.")

    X_test_sampled_list = []
    y_test_sampled_list = []

    # Iterate through each unique intent to sample examples
    for intent_class in unique_intents:
        # Filter X_test and y_test for the current intent class
        X_intent = X_test[X_test['intent'] == intent_class]
        y_intent = y_test[X_test['intent'] == intent_class] # y_test should align with X_test indices

        # Sample up to 10 examples from the current intent class
        # Use min(len(X_intent), 10) to handle classes with fewer than 10 examples
        if not X_intent.empty:
            sampled_indices = X_intent.sample(n=min(len(X_intent), 10), random_state=42).index
            X_test_sampled_list.append(X_test.loc[sampled_indices])
            y_test_sampled_list.append(y_test.loc[sampled_indices])

    # Concatenate all sampled data into new DataFrames
    X_test_sampled = pd.concat(X_test_sampled_list)
    y_test_sampled = pd.concat(y_test_sampled_list)

    print(f"\nTotal sampled examples: {len(X_test_sampled)}")
    print(f"Number of unique intents in sampled data: {X_test_sampled['intent'].nunique()}")

    print("\nHead of X_test_sampled:")
    display(X_test_sampled.head())

    print("\nValue counts for 'intent' in X_test_sampled to verify distribution:")
    display(X_test_sampled['intent'].value_counts())

    print("\nHead of y_test_sampled:")
    display(y_test_sampled.head())

Found 27 unique intent classes.

Total sampled examples: 270
Number of unique intents in sampled data: 27

Head of X_test_sampled:


,flags,category,intent,instruction_synthetic,combined_instruction
15115,BCILP,INVOICE,get_invoice,"I have lost bill #37777, could you send it to me?",[CATEGORY: INVOICE] [INTENT: get_invoice] I ha...
15617,BL,INVOICE,get_invoice,I have got to get my bill from Maria Garcia,[CATEGORY: INVOICE] [INTENT: get_invoice] I ha...
14969,BL,INVOICE,get_invoice,I don't know what I have to do to download my ...,[CATEGORY: INVOICE] [INTENT: get_invoice] I do...
15297,BLMQ,INVOICE,get_invoice,i need help to get my invoices from Maria Garcia,[CATEGORY: INVOICE] [INTENT: get_invoice] i ne...
15842,BCL,INVOICE,get_invoice,"I have got to download bill #85632, help me",[CATEGORY: INVOICE] [INTENT: get_invoice] I ha...



Value counts for 'intent' in X_test_sampled to verify distribution:


,count
intent,
get_invoice,10
contact_human_agent,10
check_refund_policy,10
contact_customer_service,10
switch_account,10
recover_password,10
check_invoice,10
set_up_shipping_address,10
place_order,10



Head of y_test_sampled:


,response
15115,Not a problem at all! I completely understand ...
15617,I'm happy to help! I understand your need to a...
14969,Sure thing! I understand your confusion about ...
15297,I'll make it happen! I completely understand y...
15842,I get it your need to download bill #{{Invoice...


Let's redefine the zero-shot prompt structure. This updated prompt will explicitly instruct the LLM to identify the intent from the customer query and provide the list of possible intent categories it should choose from. After the modification, I will show an example of the generated prompt.

In [13]:
# Get the list of all unique intent classes from the full dataset (y_train, y_val, y_test)
# This provides the complete set of labels the LLM should choose from
all_unique_intents = pd.concat([y_train, y_val, y_test]).unique()
all_unique_intents_str = ", ".join(sorted(all_unique_intents))

def create_zero_shot_prompt(instruction: str, available_intents: str) -> str:
    """Constructs a zero-shot prompt for intent classification.

    Args:
        instruction (str): The combined instruction from the dataset.
        available_intents (str): A comma-separated string of all possible intent classes.

    Returns:
        str: The complete zero-shot prompt.
    """
    prompt = f'''Please classify the intent of the following customer query. Choose one intent from the provided list:

Available Intents: {available_intents}

Customer Query: """{instruction}"""

Extracted Intent:'''
    return prompt

# Example usage with a sample from X_test_sampled
if not X_test_sampled.empty:
    sample_instruction = X_test_sampled['combined_instruction'].iloc[0]
    example_prompt = create_zero_shot_prompt(sample_instruction, all_unique_intents_str)
    print("Example Zero-Shot Prompt:\n")
    print(example_prompt)
else:
    print("X_test_sampled is empty. Cannot generate an example prompt.")

Streaming output truncated to the last 5000 lines.
3. Upgrade Option: Look for an option that says "Upgrade" or "Switch Account Type".
4. {{Account Category}} Account: Select the "{{Account Category}} Account" option from the available choices.
5. Confirm and Activate: Follow the prompts on the screen to confirm your switch to the {{Account Category}} account and activate it.

If you encounter any difficulties or have further questions, please don't hesitate to let me know. I'm here to ensure a smooth transition for you. Switching to the {{Account Category}} account will provide you with a range of exclusive benefits and features. Enjoy the enhanced experience!, Thank you for your message regarding the cancellation of your {{Account Type}} account. I understand that you would like more information about the cancellation process. Our team is here to assist you every step of the way. To provide you with accurate information, could you please share more details about the specific concerns

## Implement LLM Inference Function

### Subtask:
Provide a placeholder or example function that demonstrates how to make an LLM API call. This function will take a prompt and return the LLM's response. It will also include guidance for the user to insert their actual API key and handle the specific API integration for their chosen LLM, emphasizing careful usage to manage costs.

In [14]:
import os
import google.generativeai as genai
from google.colab import userdata

# Configure the Gemini API (if you are using Google's Gemini)
# If you are using another LLM like OpenAI GPT, you would import their client library here
try:
    # Retrieve GOOGLE_API_KEY from Colab secrets
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    if GOOGLE_API_KEY:
        genai.configure(api_key=GOOGLE_API_KEY)
        print("Gemini API configured successfully.")
    else:
        print("Warning: GOOGLE_API_KEY not found. Please ensure it's set in Colab secrets with the name 'GOOGLE_API_KEY'.")
except Exception as e:
    print(f"Error configuring Gemini API: {e}")

def get_llm_prediction(prompt: str, model_name: str = 'gemini-2.0-flash', timeout_seconds: int = 60) -> str:
    """Makes an API call to the specified LLM and returns its response.

    Args:
        prompt (str): The zero-shot prompt to send to the LLM.
        model_name (str): The name of the LLM model to use (e.g., 'gemini-2.0-flash', 'gpt-3.5-turbo').
        timeout_seconds (int): Maximum time in seconds to wait for an API response.

    Returns:
        str: The extracted intent from the LLM's response, or an error message.
    """
    try:
        if model_name.startswith('gemini'):
            # Example for Google Gemini
            model = genai.GenerativeModel(model_name)
            response = model.generate_content(prompt, request_options={'timeout': timeout_seconds})
            # Assuming the LLM is expected to output just the intent label
            return response.text.strip()
        # Add conditions for other models here (e.g., for OpenAI GPT, Anthropic Claude)
        # elif model_name.startswith('gpt'):
        #     from openai import OpenAI
        #     client = OpenAI(api_key=OPENAI_API_KEY)
        #     response = client.chat.completions.create(
        #         model=model_name,
        #         messages=[{"role": "user", "content": prompt}],
        #         max_tokens=50 # Limit response length to get just the intent
        #     )
        #     return response.choices[0].message.content.strip()
        else:
            return "Error: Unsupported model_name. Please update the inference function."
    except Exception as e:
        return f"LLM Inference Error: {e}"

# Example of how to use the function (will be executed in the next step)
# You would typically iterate through your X_test_sampled here.
# test_prompt = create_zero_shot_prompt("I want to cancel my order", all_unique_intents_str)
# prediction = get_llm_prediction(test_prompt)
# print(f"\nTest Prediction: {prediction}")

Gemini API configured successfully.


## Run LLM Inference and Collect Predictions

### Subtask:
Iterate through the sampled test data, generate a zero-shot prompt for each entry, call the LLM inference function to get a prediction, and collect all predicted intents. This step will also include parsing the LLM's response to extract the intent label.

In [ ]:
import time

# Initialize a list to store LLM predictions
llm_predictions = []

print(f"Starting LLM inference for {len(X_test_sampled)} samples...")

# Iterate through each sample in X_test_sampled
for index, row in X_test_sampled.iterrows():
    instruction = row['combined_instruction']

    # Create the zero-shot prompt
    prompt = create_zero_shot_prompt(instruction, all_unique_intents_str)

    # Get prediction from LLM, with a 60-second timeout
    prediction = get_llm_prediction(prompt, model_name='gemini-2.0-flash', timeout_seconds=60)

    llm_predictions.append(prediction)

    # Use a very small delay given the high paid-tier limits
    time.sleep(0.1)

print("LLM inference completed.")

# Add the predictions to a new DataFrame or Series for easy comparison
predictions_df = pd.DataFrame({'llm_predicted_intent': llm_predictions}, index=X_test_sampled.index)

print("\nHead of LLM Predictions:")
display(predictions_df.head())

print("\nValue counts of LLM Predictions:")
display(predictions_df['llm_predicted_intent'].value_counts())

Starting LLM inference for 270 samples...
